In [175]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib qt

In [176]:
# Parameters definition

c = 3e8
pi = np.pi
f = 1e10

t = np.linspace(0, 100/f, 100)
exp = np.exp(1j * 2 * pi * f * t)

e0 = 8.85e-12
mu0 = 4*pi*1e-7
l0 = 15e-2
n0 = 1
eta0 = 377
z0 = np.linspace(0, l0, 1500)

n1 = 3
e1 = n1**2 * e0
l1 = 10e-2 
z1 = np.linspace(0, l1, 1000)
eta1 = eta0 / n1

n2 = 1.5
e2 = n2**2 * e0
l2 = 5e-2
z2 = np.linspace(0, l2, 500)
eta2 = eta0 / n2

n3 = 1
e3 = n3**2 * e0
l3 = 15e-2
z3 = np.linspace(0, l3, 1500)
eta3 = eta0 / n3

In [177]:
def compute_rho_tau(n1, n2):
    rho = (n1 - n2) / (n1 + n2)
    tau = 2*n1 / (n1+n2)
    return rho, tau

def propagation(Ef0, Eb0, n, z):
    k = 2 * pi * f * n / c
    Ef = np.exp(1j * k * np.flip(z)) * Ef0
    Eb = np.exp(-1j * k * np.flip(z)) * Eb0
    return Ef, Eb

def interface(rho, tau, Efp , Ebp):
    Ef = (1/tau) * (Efp + rho*Ebp)
    Eb = (1/tau) * (rho*Efp + Ebp)
    return Ef, Eb


In [178]:
Ef3 = np.zeros(len(z3), dtype='complex')
Ef3, Eb3 = propagation(1, 0, n3, z3)               # Here, -z because we go from left to infinity

Ef2 = np.zeros(len(z2), dtype='complex')
Eb2 = np.zeros(len(z2), dtype='complex')
rho3, tau3 = compute_rho_tau(n2, n3)
Efint, Ebint = interface(rho3, tau3, Ef3[0], Eb3[0])
Ef2, Eb2 = propagation(Efint, Ebint, n2, z2)    # +z because from right to left

print(Efint)
print(Ef2[-1])


Ef1 = np.zeros(len(z1), dtype='complex')
Eb1 = np.zeros(len(z1), dtype='complex')
rho2, tau2 = compute_rho_tau(n1, n2)
Efint, Ebint = interface(rho2, tau2, Ef2[0], Eb2[0])
Ef1, Eb1 = propagation(Efint, Ebint, n1, z1)    # +z because from right to left

Ef0 = np.zeros(len(z0), dtype='complex')
Eb0 = np.zeros(len(z0), dtype='complex')
rho1, tau1 = compute_rho_tau(n0, n1)
Efint, Ebint = interface(rho1, tau1, Ef1[0], Eb1[0])
Ef0, Eb0 = propagation(Efint, Ebint, n0, z0)    # +z because from right to left

E0 = (Ef0 + Eb0)
E1 = (Ef1 + Eb1)
E2 = (Ef2 + Eb2)
E3 = (Ef3 + Eb3)


Etot = np.concatenate((E0, E1, E2, E3))
ztot = np.linspace(-l0, l1+l2+l3, 4500)

# plt.plot(ztot, np.real(Etot))
# plt.show()

(0.8333333333333334-3.981133731623212e-15j)
(0.8333333333333334-3.981133731623212e-15j)


In [179]:
Evstime = np.zeros((len(t), len(Etot)))
for i in range(len(t)):
    Evstime[i] = np.real(Etot * exp[i])

# plt.plot(ztot, np.real(Evstime[0]))
# plt.show()

In [180]:
fig, ax = plt.subplots()
ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.set_title("Field in layered media")
ax.grid()
ax.set_ylim(-3, 3)

line, = ax.plot(ztot, Evstime[0], label="Non dispersive medium")
ax.legend()

def anim(i):       
    line.set_ydata(Evstime[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=50, repeat=True ,blit=False)
plt.show()

# writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'),bitrate=1800)
# ani.save('layered2.gif', writer=writer)